# Weekly Project 5
## Global Registration implementation.
## Task 1
Today your project is to implement a global registration algorithm.

It should be able to roughly align two pointclouds.
1. Implement global registration
2. Can you fit **r1.pcd** and **r2.pcd**?
3. Can you fit **car1.ply** and **car2.ply**?
These are in the *global_registration* folder



In [21]:
import open3d as o3d
import numpy as np
import copy

# helper function for drawing if you want it to be more clear which is which set recolor=True
def draw_registrations(source, target, transformation = None, recolor = False):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    if(recolor):
        source_temp.paint_uniform_color([1, 0.706, 0])
        target_temp.paint_uniform_color([0, 0.651, 0.929])
    if(transformation is not None):
        source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp])

In [41]:
def global_alignment(source, target, voxel_size=0.05, corr_length=0.9,
                     radius_scale=2, max_nn=100, distance_threshold_scale=2, normal_checker=0.95,
                     ransac_stopping_criteria=100000):
    #####################################
    # Downsample and find features here #
    #####################################
    # 1 Downsample both point clouds
    source_down = source.voxel_down_sample(voxel_size=voxel_size)
    target_down = target.voxel_down_sample(voxel_size=voxel_size)

    # 2 Estimate surface normals of pointclouds
    source_down.estimate_normals()
    target_down.estimate_normals()

    # 3 FPFH feature descriptor and correspondance on the downsampled pointcloud. 
    # Raius used to describe a feature
    radius_feature = voxel_size * radius_scale
    fpfh_source = o3d.pipelines.registration.compute_fpfh_feature(source_down,
        search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=max_nn))

    fpfh_target = o3d.pipelines.registration.compute_fpfh_feature(target_down,
        search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=max_nn))

    ##########
    # RANSAC #
    ##########
    # We can try both point-to-point or point-to plane
    point_to_point =  o3d.pipelines.registration.TransformationEstimationPointToPoint(False)
    distance_threshold = voxel_size * distance_threshold_scale
    c0 = o3d.pipelines.registration.CorrespondenceCheckerBasedOnEdgeLength(corr_length)
    c1 = o3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(distance_threshold)
    c2 = o3d.pipelines.registration.CorrespondenceCheckerBasedOnNormal(normal_checker)

    checker_list = [c0,c1,c2]
    
    result = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
        source_down,
        target_down,
        fpfh_source, 
        fpfh_target,
        True, # mutual_filter
        voxel_size*2,
        point_to_point, # point_to_point or point_to_plane
        checkers = checker_list,
        criteria=o3d.pipelines.registration.RANSACConvergenceCriteria(ransac_stopping_criteria, 0.999)
        )
    return result

In [42]:
r1 = o3d.io.read_point_cloud("global_registration/r1.pcd")
r2 = o3d.io.read_point_cloud("global_registration/r2.pcd")
ransac_result_r1_r2 = global_alignment(r1, r2, 
                    voxel_size=0.05, corr_length=0.9,
                    radius_scale=2, max_nn=100, distance_threshold_scale=2, normal_checker=0.95,
                    ransac_stopping_criteria=100000)

draw_registrations(r1, r2, ransac_result_r1_r2.transformation, True)

In [37]:
r1 = o3d.io.read_point_cloud("global_registration/car1.ply")
r2 = o3d.io.read_point_cloud("global_registration/car2.ply")

draw_registrations(r1, r2, recolor=True)

ransac_result_r1_r2 = global_alignment(r1, r2, 
                    voxel_size=0.05, corr_length=0.9,
                    radius_scale=2, max_nn=100, distance_threshold_scale=2, normal_checker=0.95,
                    ransac_stopping_criteria=100000)
                    
draw_registrations(r1, r2, ransac_result_r1_r2.transformation, True)



## Task 2 (Challange)
Challanges attempt either or both:
- Implement local registration.

- Attempt to reconstruct the car from the images in *car_challange* folder.

You can use the exercises from monday as a starting point.